## 긍정, 부정의 감성분석 : 데이터 양을 늘리기

In [ ]:
import pandas as pd

# 두 파일을 병합하기
df_train = pd.read_table('ratings_train.txt')
df_test = pd.read_table('ratings_test.txt')

df = pd.concat([df_train, df_test])

In [ ]:
# 중복 제거
df.drop_duplicates(subset=['document'], inplace=True)
# 결측치 제거
df = df.dropna(how='any')

In [ ]:
# 중복과 결측치를 제거한 데이터를 파일로 저장하기
df.to_csv('naver_filtered.csv')

In [ ]:
df = pd.read_csv("naver_filtered.csv")
# 데이터를 라벨 0,1의 비중을 맞춰서 1000를 추출

df = df[['document','label']]

df_pos = df[df['label'] == 1]
df_neg = df[df['label'] == 0]

sample_pos = df_pos.sample(n=1500, random_state=12)
sample_neg = df_neg.sample(n=1500, random_state=12)

sample_df = pd.concat([sample_pos,sample_neg], ignore_index=True)

print("샘플링된 데이터 개수 : ", len(sample_df))
print("라벨별 데이터 개수 : \n", sample_df['label'].value_counts())

df = sample_df

In [ ]:
# 한글 형태소 분석기를 통해, 영화 리뷰 문장들을 전처리
# 한글만 남기고 모두 제거하기
df['document'] = df['document'].str.replace("[^가-힣 ]","",regex=True)
# '빈칸'만 남은 데이터는 공백으로 변경
df['document'] = df['document'].str.replace("^ +","",regex=True)
# 공백만 있는 데이터는 Null / Nan으로 변경
import numpy as np
df['document'] = df['document'].replace("", np.nan)

In [ ]:
# df에서 null 제거
df = df.dropna(how='any')
print(df.info())

In [ ]:
from konlpy.tag import Okt
okt = Okt()
stopwords =  ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '.']

# 훈련 데이터 만들기
train_data = []
for i in range(len(df)):
    emotion = df.iloc[i]['label']
    text = df.iloc[i]['document']
    # 문장을 형태소 분석하기 -> 문장을 단어(토큰)의 리스트로 변환 
    tokenized_text = okt.morphs(text)
    # 형태소 분석결과에서 불용어를 제거함
    filtered_tokens = [ word for word in tokenized_text if not word in stopwords ]
    # (토큰화 된 문장, 감정) 형태의 튜플을 리스트에 저장
    train_data.append((filtered_tokens,emotion))

In [ ]:
all_tokens = []
for tokens, emotion in train_data :
    for token in tokens :
        all_tokens.append(token)
vocabulary = list(set(all_tokens))

In [ ]:
def create_feature_set(tokens, vocabulary) :
    token_set = set(tokens)
    features = { word : (word in token_set) for word in vocabulary }
    return features

In [ ]:
train_feature_datas = []
train_feature_datas = [
    (create_feature_set(tokens,vocabulary),emotion)
    for tokens, emotion in train_data ]

In [ ]:
import nltk
model = nltk.NaiveBayesClassifier.train(train_feature_datas)

In [ ]:
def classify_text(text, model, tokenizer, vocabulary) :
    tokens = tokenizer.morphs(text)
    filtered_tokens = [ word for word in tokens if not word in stopwords ]
    features = create_feature_set(filtered_tokens, vocabulary)
    clasified_label = model.classify(features)
    print(model.prob_classify(features).prob(clasified_label))
    result = "부정" if clasified_label == 0 else "긍정"
    print("예측 결과 : ", result)
    return result

In [ ]:
text_list = [
    "새로운 직장 동료들이 다들 좋은 사람들이라 편하게 적응하고 있어",   #기쁨
    "첫 발표인데, 갑자기 노트북이 꺼져서 정말 당황스러웠어",            #당황
    "상사가 자기가 할 일을 나에게 다 떠넘겨서 너무 화가 나",            #분노
    "내일 중요한 면접이 있는데 하나도 준비를 못 해서 너무 걱정돼",      #불안
    "믿었던 친구에게 배신당해서 큰 상처를 받았어",                      #상처
    "열심히 준비한 프로젝트에서 해고 통보를 받으니 너무 슬프다"         #슬픔
]
for text in text_list :
    classify_text(text, model, okt, vocabulary)

In [ ]:
text_list = [
    "오늘 본 영화가 정말 감동적이었고, 강아지가 너무 귀여웠어.",        #기쁨
    "친구가 갑자기 내 생일 파티를 열어줘서 깜짝 놀랐어!",               #당황
    "밤새 모기가 윙윙거려서 잠을 설쳤더니 짜증 나.",                    #분노
    "내일 소개팅하는데 무슨 말을 해야 할지 모르겠어.",                  #불안
    "다이어트에 또 실패해서 속상해.",                                   #상처
    "키우던 반려 식물이 시들어버려서 마음이 아파."                      #슬픔
]
for text in text_list :
    classify_text(text, model, okt, vocabulary)

In [ ]:
text_list = [
    "이 영화 진짜 역대급 꿀잼! 배우들 연기도 최고였어요.",
    "시간이 아깝다... 스토리가 너무 지루하고 재미없어요",
    "그냥 평범한 영화, 볼만은 한데 추천할 정도는 아님"
]
for text in text_list :
    classify_text(text, model, okt, vocabulary)